In [57]:
import pandas as pd
import re
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as stp
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
df = pd.read_csv('/content/drive/MyDrive/FeatureEngineering/Res_data.csv')
df_cp = df.copy()

In [59]:
df.head()

,Name,Email,Mobile_Number,Skills,Education,Experience
0,Prasad Nagar,"['nathan.johnson567@example.com', 'liam.jones7...","['1234567890', '2345678901']","['Python', 'Java', 'JavaScript', 'SQL', 'CSS']","('BTECH','2020')",6
1,AKSHAY DESHBHRATAR,"['hannah.davis890@gmail.com', 'jackson.thompso...","['3456789012', '4567890123']","['HTML', 'C++', 'C#', 'Ruby', 'PHP']","('BTECH','2020')",5
2,kaashiv infotech,"['olivia.martinez456@yahoo.com', 'abigail.whit...","['5678901234', '6789012345']","['Swift', 'Objective-C', 'React', 'Angular', '...","('BTECH','2020')",3
3,Aaditya Singh,"['ethan.jackson345@example.com', 'sophia.mille...","['7890123456', '8901234567']","['Express.js', 'Django', 'Flask', 'Vue.js', 'B...","('BTECH','2020')",1
4,Aakash Singh,"['avery.anderson123@hotmail.com', 'mia.wilson4...","['9012345678', '0123456789']","['MongoDB', 'MySQL', 'PostgreSQL', 'SQLite', '...","('BTECH','2020')",7


In [60]:
df.isnull().sum()

Name             0
Email            0
Mobile_Number    0
Skills           0
Education        0
Experience       0
dtype: int64

In [61]:
df['Name'].fillna('Unknown', inplace=True)
df['Email'].fillna('missing@example.com', inplace=True)
df['Mobile_Number'].fillna('0000000000', inplace=True)
df['Skills'].fillna('No Skills', inplace=True)
df['Education'].fillna('Unknown', inplace=True)
df['Experience'].fillna(0, inplace=True)

In [62]:

all_degrees = []

for degree in df['Education']:
    if pd.notna(degree):
        # Extract only the degree name by removing year and spaces/dots
        degree_name = re.sub("[\s.\d]", "", str(degree)).upper()
        all_degrees.append(degree_name)

# Convert the list to a set to get unique degree names
unique_degrees = set(all_degrees)

print(unique_degrees)


{"('BTECH','')"}


In [63]:
df.Experience.value_counts()

5    4
3    4
1    4
7    4
6    3
Name: Experience, dtype: int64

In [64]:
ind = 0
for i in df.Experience:
  if i < 0:
    df.loc[ind,'work_experience'] = i*-1
  elif i > 40:
    df.loc[ind,'work_experience'] = 0
  ind +=1
df['Experience'] = df['Experience'].astype(int)

In [65]:
lemmatizer = WordNetLemmatizer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in analyzer(doc) if w not in set(stp.words('english')))

In [66]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [67]:
import numpy as np
def get_tf_idf_cosine_similarity(job_desc, all_resumes):
    tf_idf_vect = TfidfVectorizer(analyzer=stemmed_words)
    tf_idf_desc_vector = tf_idf_vect.fit_transform([job_desc])
    tf_idf_resume_vector = tf_idf_vect.transform(all_resumes)

    cosine_similarity_list = []
    for i in range(tf_idf_resume_vector.shape[0]):
        desc_vector = np.asarray(tf_idf_desc_vector.todense())
        resume_vector = np.asarray(tf_idf_resume_vector[i].todense())
        cosine_similarity_list.append(cosine_similarity(desc_vector, resume_vector)[0][0])

    return cosine_similarity_list

In [68]:
with open('/content/drive/MyDrive/FeatureEngineering/JD_fullstack.txt','r',encoding = 'utf-8') as f:
  file_desc_lst =  [r.replace('\n', '') for r in f.readlines()]

In [69]:
job_description = ''

for i in file_desc_lst:
  if len(job_description) == 0:
    job_description = str(i)
  else:
    job_description = job_description + ' ' + str(i)

In [70]:
job_description

"Job Title: Full Stack Web Developer  We are looking for a talented Full Stack Web Developer to join our dynamic team. The ideal candidate will have a strong foundation in both front-end and back-end development, with a passion for creating responsive and user-friendly web applications.  Responsibilities: - Collaborate with the design team to implement user-friendly interfaces using HTML, CSS, and JavaScript (React, Angular, or Vue.js). - Develop robust server-side logic using Node.js, Express.js, and databases like MongoDB or MySQL. - Write clean, efficient, and maintainable code following industry best practices. - Ensure the responsiveness and cross-browser compatibility of web applications. - Collaborate with the DevOps team to deploy applications using Docker and Kubernetes. - Integrate RESTful APIs and GraphQL to fetch and update data efficiently. - Optimize application performance, troubleshoot issues, and perform debugging. - Participate in code reviews and provide constructive

In [71]:
all_resume_text = []

for i in df.iloc[:].values:
  s = ''
  for j in list(i):
    if len(s) == 0:
      s = str(j)
    else:
      s = s + ' , ' + str(j)
  all_resume_text.append(s)

In [72]:
all_resume_text

["Prasad Nagar , ['nathan.johnson567@example.com', 'liam.jones789@outlook.com'] , ['1234567890', '2345678901'] , ['Python', 'Java', 'JavaScript', 'SQL', 'CSS'] , ('BTECH','2020') , 6",
 "AKSHAY DESHBHRATAR , ['hannah.davis890@gmail.com', 'jackson.thompson234@hotmail.com'] , ['3456789012', '4567890123'] , ['HTML', 'C++', 'C#', 'Ruby', 'PHP'] , ('BTECH','2020') , 5",
 "kaashiv infotech , ['olivia.martinez456@yahoo.com', 'abigail.white890@gmail.com'] , ['5678901234', '6789012345'] , ['Swift', 'Objective-C', 'React', 'Angular', 'Node.js'] , ('BTECH','2020') , 3",
 "Aaditya Singh , ['ethan.jackson345@example.com', 'sophia.miller678@yahoo.com'] , ['7890123456', '8901234567'] , ['Express.js', 'Django', 'Flask', 'Vue.js', 'Bootstrap'] , ('BTECH','2020') , 1",
 "Aakash Singh , ['avery.anderson123@hotmail.com', 'mia.wilson456@gmail.com'] , ['9012345678', '0123456789'] , ['MongoDB', 'MySQL', 'PostgreSQL', 'SQLite', 'Git'] , ('BTECH','2020') , 7",
 "Aashish Kumar , ['logan.taylor789@example.com', 

In [73]:
import nltk

In [74]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [75]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [76]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [77]:
cos_sim_list = get_tf_idf_cosine_similarity(job_description, all_resume_text)

In [78]:
zipped_resume_rating = zip(cos_sim_list,df_cp.Name,[x for x in range(len(df))])
sorted_resume_rating_list = sorted(zipped_resume_rating, key = lambda x: x[0], reverse=True)

In [79]:
resume_score = [round(x*100,2) for x in cos_sim_list]
pd.concat([df_cp.Name, pd.DataFrame(resume_score, columns=['resume_score(%)'])], axis=1).sort_values(by=['resume_score(%)'],ascending=False)

,Name,resume_score(%)
3,Aaditya Singh,33.73
18,Aditya Pande,32.33
8,Abdeali Tin,32.33
16,Adarsh Kocherla,32.33
2,kaashiv infotech,30.98
6,Aastha Patel,20.78
4,Aakash Singh,14.91
0,Prasad Nagar,14.61
1,AKSHAY DESHBHRATAR,10.33
15,Adarsh CHATURVEDI,10.33


In [80]:
top_10_resumes = pd.concat([df_cp.Name, pd.DataFrame(resume_score, columns=['resume_score(%)'])], axis=1).nlargest(10, 'resume_score(%)')
top_10_resumes.reset_index(drop=True, inplace=True)
top_10_resumes.index += 1
top_10_resumes

,Name,resume_score(%)
1,Aaditya Singh,33.73
2,Abdeali Tin,32.33
3,Adarsh Kocherla,32.33
4,Aditya Pande,32.33
5,kaashiv infotech,30.98
6,Aastha Patel,20.78
7,Aakash Singh,14.91
8,Prasad Nagar,14.61
9,AKSHAY DESHBHRATAR,10.33
10,Adarsh CHATURVEDI,10.33
